# Decoding of Next-Token Predictions

This note book is designed to reimplement Pathscope **section 1:Next-Token Prediction**

In [2]:
print("hello world")

hello world


In [3]:
import torch
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cup')

In [4]:
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return the number of available GPUs
print(torch.cuda.get_device_name(0))  # Should return the name of the GPU


True
1
NVIDIA GeForce GTX 1080 Ti


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM,DataCollatorWithPadding
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.amp import autocast
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# Your code here
# import dataloader


In [6]:
model_dict = ["Qwen/Qwen2-0.5B","Qwen/Qwen2.5-0.5B-Instruct","Qwen/Qwen2-1.5B"]

In [7]:
model_num = 0

tokenizer = AutoTokenizer.from_pretrained(model_dict[model_num])
model = AutoModelForCausalLM.from_pretrained(model_dict[model_num]).to(device)
model.eval()


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [8]:
# model(tokenizer('how are you doing today?'))

In [9]:
print(model.__class__.__module__)
print(model.config.model_type)

transformers.models.qwen2.modeling_qwen2
qwen2


In [10]:
type(model)

transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM

In [11]:
type(tokenizer)

transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast

In [ ]:
from transformers import Qwen2ForCausalLM
model2 = Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B')

In [13]:
# print(model2)

In [14]:
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [15]:
# define a function to get hidden states of each layer of the model

def get_hidden_state(model, input_ids):
    with torch.no_grad():
        outputs = model(input_ids.to(device), output_hidden_states=True)
    return outputs.hidden_states[1:]  #Size([batch_size, num_layers, sequence_length, hidden_size])




In [16]:
# load the Pile data "https://huggingface.co/datasets/EleutherAI/pile" from hugging face, unfortunately it is not available
# use the following uncoprighted pile dataset instead
# https://huggingface.co/datasets/monology/pile-uncopyrighted

dataPile= load_dataset("monology/pile-uncopyrighted",split='train',streaming=True)


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [17]:
# shuffle the dataset before spliting them 
# reference resource "https://huggingface.co/docs/datasets/v3.3.2/stream"

dataPile = dataPile.shuffle(seed=50)
training_data = dataPile.take(10000)
# train : val = 5 : 1
validation_data = dataPile.skip(10000).take(2000)

In [18]:
list(validation_data)[:4]

[{'text': 'The present invention relates generally to the art of sputtering silicon-containing target materials in a reactive atmosphere, and more particularly to reactive sputtering of silicon targets further comprising a transition metal.\nGerman Patent Application DE 3,906,453 Al to Szczybowski et al. discloses sputtering silicides, such as nickel silicide (NiSi.sub.2), in an oxidizing atmosphere to deposit dielectric oxide films.\nThe present invention involves silicon-nickel alloys ranging from 3 to 18 weight percent nickel. Targets of silicon-nickel alloys are sputtered in an atmosphere comprising nitrogen, oxygen, inert gases and mixtures thereof to produce silicon-nickel containing coatings including oxides, nitrides and oxynitrides. The silicon-nickel compositions of the present invention comprise sufficient nickel to provide target stability and a desirable sputtering rate while keeping the absorption of the resulting films relatively low.',
  'meta': {'pile_set_name': 'USPTO

In [19]:
text = "1. Field of the Invention\nThe present invention relates to toothbrushes and, in particular, to a toothbrush having a hollow handle defining a paste-holding cavity wherein toothpaste is forcibly dispensed therefrom"

In [20]:
tokenizer([text,'hee asdf po afsdf '], return_tensors="pt",max_length=21, truncation=True, padding="max_length")

{'input_ids': tensor([[    16,     13,   8601,    315,    279,    758,   7459,    198,    785,
           3042,  27130,  35616,    311,  25507,  36061,    288,    323,     11,
            304,   3953,     11],
        [ 49053,    438,   2940,   3193,  66713,   2940,    220, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [21]:

import random

def preprocess(example):

    text = example['text']
    inputs = tokenizer(text, return_tensors='pt', max_length=50, truncation=True) # dict_keys(['input_ids', 'attention_mask'])
    inputs_ids = inputs['input_ids'][0] # becasue the batch size is 1 

    if len(inputs_ids) > 5:
        trim_len = random.randint(5,  len(inputs_ids) -1)   # to introduce randomness in the training data
        inputs_ids = inputs_ids[:trim_len]

    return {'input_ids': inputs_ids, 'attention_mask': inputs['attention_mask'][0][:len(inputs_ids)]}

In [22]:
training_data = training_data.map(preprocess,remove_columns=['text','meta'])
validation_data = validation_data.map(preprocess,remove_columns=['text','meta'])

In [23]:
next(iter(training_data))

{'input_ids': tensor([   35,   321,  1345,   496,   220,    16,    24,    24,    23,   271,
         38704, 10099,   279,  2480,  5700, 52767,  1345,   496,  2860,  1283,
           279,  4285, 12227]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [24]:
# design a collate function 

In [25]:
# we can change the attentino mask to focus on the token of interest

In [26]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
# PyTorch
torch.cuda.memory_summary()

# Or more detailed
torch.cuda.memory_allocated()
torch.cuda.memory_reserved()


2097152000

## 1. Logit lens

In [ ]:


def logit_len_for_test(hidden_states,refernce_logits):
    pass
    


def logit_len(hidden_states,mask,refernce_logits):
    """

    hidden_states: Tuple[Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model

    mask: Size([batch_size, sequence_length])

    refernce_logits: Size([batch_size, sequence_length, vocab_size]): this will be converted to Size([batch_size, vocab_size])
    
    """
    W_U = model.lm_head.weight
    
    # using attention mask to get the last token of the sequence

    num_layer  = len(hidden_states)
    batch_size, sequence_length, hidden_size = hidden_states[0].shape

    last_token_pos = torch.sum(mask,dim=1) # Size([batch_size])

    reference_logits = refernce_logits[torch.arange(len(mask)),last_token_pos -1 ,:] # Size([batch_size, vocab_size])
    reference_logits_max_token = torch.argmax(reference_logits,dim = -1)


    for each_layer in hidden_states:
        layer_hidden = each_layer[torch.arange(batch_size),last_token_pos-1] # Size([batch_size, hidden_size])
        # layer_hidden = layer_hidden[torch.arange(batch_size),:] # Size([batch_size, hidden_size])
        logits = torch.einsum('bd,vd->bv', layer_hidden, W_U)  # Size([batch_size, vocab_size])
        logits_max_token = torch.argmax(logits,dim = -1)
        print(torch.sum(logits_max_token == reference_logits_max_token))    


    return  



    # layer_indics = torch.arange(num_layer,device=hidden_states[0].device).view(num_layer,1)
    # batch_indics = torch.arange(batch_size, device=hidden_states[0].device).view(1,batch_size)

    # layer_indics = layer_indics.expand(num_layer,batch_size)
    # batch_indics = batch_indics.expand(num_layer,batch_size)
    # position = last_token_pos.view(1,batch_size).expand(num_layer,batch_size)

    # result_hidden = hidden_states[layer_indics,batch_indics,position,:] # Size([layer, batch_size, hidden_size])

    # logits = torch.einsum('bld,vd->blv', result_hidden, W_U)  # Size([layer, batch_size, vocab_size])

   





    # resutls_token_hidden = []
    # for layer in hidden_states:
    #     for i, b in enumerate(layer):
    #         last_token_hidden = b[last_token_pos[i] - 1]
    #         resutls_token_hidden.append(last_token_hidden)

    # resutls_token_hidden = torch.tensor(resutls_token_hidden).reshape(len(hidden_states),len(mask),hidden_states[0].shape[-1]) # Size([layer, batch_size, hidden_size])
    # logits = torch.einsum('bld,vd->blv', resutls_token_hidden, W_U)  # Size([layer, batch_size, vocab_size])

    # return  torch.argmax(reference_logits,dim = -1) == torch.argmax(logits,dim=-1)

batch_size = 32

i = 0
for batch in DataLoader(validation_data, batch_size=batch_size, collate_fn=data_collator):
    # print(batch)k
    batch.to(device)
    with torch.no_grad():
        model_outputs = model(**batch,output_hidden_states=True)
        hidden_states = model_outputs.hidden_states[1:]  # exclude the embedding layer
        reference_logits = model_outputs.logits
        mask = batch['attention_mask']

    # reference_logits = reference_logits[torch.arange(len(mask)),torch.sum(mask,dim=1)] # Size([batch_size, vocab_size])
    Compare_logit = logit_len(hidden_states,mask,reference_logits) 
    # break
    i += 1
    print('one batch done')

    # if i >= 5:
        # break

In [ ]:
# Clear GPU cache
torch.cuda.empty_cache()

In [ ]:
print(# PyTorch
torch.cuda.memory_summary())

# Or more detailed
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

In [ ]:
! nvidia-smi

#### Metric 
1. Precision@1
2. Suprisal 

In [ ]:
print(model_outputs['logits'].shape)
print(model_outputs['hidden_states'][4].shape)



In [ ]:
len(model_outputs['hidden_states'])

## 2. Tuned Lens

In [ ]:
from sklearn.linear_model import LinearRegression

import numpy as np


config = model.config
num_layers = config.num_hidden_layers

X_train = {l : [] for l in range(num_layers)}
Y_train = []  # finaly layer logits



batch_size = 32

def get_last_token_hidden_states(hidden_states,mask) -> None:
    """
    hiddenstates: Tuple[Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model
    """

    last_token_pos = torch.sum(mask,dim=1) # Size([batch_size])

    for i, layer in enumerate(hidden_states):
        X_train[i].append(layer[torch.arange(len(mask)),last_token_pos -1 ,:].cpu().numpy())   # Size([batch_size, hidden_size])

    Y_train.append(hidden_states[-1][torch.arange(len(mask)),last_token_pos -1 ,:].cpu().numpy())   # Size([batch_size, hidden_size]))

# to get the hidden states of each sequence
for batch in DataLoader(training_data, batch_size=batch_size, collate_fn=data_collator):
    batch.to(device)

    with torch.no_grad():
        model_outputs = model(**batch,output_hidden_states=True)
    batch.to('cpu')
    hidden_states = model_outputs.hidden_states[1:] # exclude the embedding layer
    mask = batch['attention_mask'].cpu()
    reference_logits = model_outputs.logits

    get_last_token_hidden_states(hidden_states,mask)

    del batch, model_outputs
    torch.cuda.empty_cache()  # Clear GPU cache



In [ ]:
X_train[0][0]

In [ ]:
len(Y_train)

In [ ]:
# for each layer,train a affing transformation to predict the final layer logits

Y_train = np.concatenate(Y_train,axis=0) # Size([num_samples, hidden_size])

affine_map_house = []

for i in range(num_layers):
    affine_map = LinearRegression(fit_intercept=True)
    X_train[i] = np.concatenate(X_train[i],axis=0) # Size([num_samples, hidden_size])
    affine_map.fit(X_train[i],Y_train)
    affine_map_house.append(affine_map)
    # break
   
    

In [ ]:

# do the same thing as logit_len but with the affine transformation bewteen the hidden states of each layer and the final layer logits


def tuned_len(hidden_states,mask,refernce_logits,model = model):
    """

    hidden_states: Tuple[Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model

    mask: Size([batch_size, sequence_length])

    refernce_logits: Size([batch_size, sequence_length, vocab_size]): this will be converted to Size([batch_size, vocab_size])
    
    """
    W_U = model.lm_head.weight
    
    # using attention mask to get the last token of the sequence

    num_layer  = len(hidden_states)
    batch_size, sequence_length, hidden_size = hidden_states[0].shape

    last_token_pos = torch.sum(mask,dim=1) # Size([batch_size])

    reference_logits = refernce_logits[torch.arange(len(mask)),last_token_pos -1 ,:] # Size([batch_size, vocab_size])
    reference_logits_max_token = torch.argmax(reference_logits,dim = -1)


    for i , each_layer in enumerate(hidden_states):
        layer_hidden = each_layer[torch.arange(batch_size),last_token_pos-1] # Size([batch_size, hidden_size]

        #***************************************************************
        # the only changes compared to logit_len
        affine_map = affine_map_house[i]   #这里竟然没有报错，当我只有第一个layer的affine map的时候
        layer_hidden_mapped = affine_map.predict(layer_hidden.cpu().numpy())
        layer_hidden_mapped = torch.tensor(layer_hidden_mapped).to(device)

        #***************************************************************
        logits = torch.einsum('bd,vd->bv', layer_hidden_mapped, W_U)  # Size([batch_size, vocab_size])
        logits_max_token = torch.argmax(logits,dim = -1)
        print(torch.sum(logits_max_token == reference_logits_max_token))    


batch_size = 32

i = 0
for batch in DataLoader(validation_data, batch_size=batch_size, collate_fn=data_collator):
    # print(batch)k
    batch.to(device)
    with torch.no_grad():
        model_outputs = model(**batch,output_hidden_states=True)
        hidden_states = model_outputs.hidden_states[1:]  # exclude the embedding layer
        reference_logits = model_outputs.logits
        mask = batch['attention_mask']

    # reference_logits = reference_logits[torch.arange(len(mask)),torch.sum(mask,dim=1)] # Size([batch_size, vocab_size])
    Compare_logit = tuned_len(hidden_states,mask,reference_logits) 
    # break
    i += 1
    print('one batch done')

    if i >= 5:
        break

In [ ]:
X_train[0].shape

In [ ]:
Y_train.shape

we can conclude that tuned lens really outputperform vallina logit lean

## 3.Token Identitiy Patchscope 

set the hidden state at the specic layer and position to the last 

### A. Construct **Few-Shot Token Identity Prompts**

In [28]:
# sample a random set of k tokens for all the models, where k was also randomly sampled from the interval [1, . . . , 10].

# to construct 5 realization of token iDs series of variable length  



def craft_realization() -> torch.Tensor:
    """
    return: Size([num_realization, num_tokens])
    """

    identity_prompts = []

    num_realization = 5
    
    for j in range(num_realization):
        num_tokens = random.sample(range(tokenizer.vocab_size),random.randint(1,10))
        prompt = ';'.join([ f"{tokenizer.decode([i])} -> {tokenizer.decode([i])}"  for i in num_tokens]) + '; ?'
        identity_prompts.append(prompt)


    return identity_prompts

identity_token_prompts = craft_realization()




In [27]:
print(identity_token_prompts)

NameError: name 'identity_token_prompts' is not defined

In [30]:
model.model.layers

ModuleList(
  (0-23): 24 x Qwen2DecoderLayer(
    (self_attn): Qwen2Attention(
      (q_proj): Linear(in_features=896, out_features=896, bias=True)
      (k_proj): Linear(in_features=896, out_features=128, bias=True)
      (v_proj): Linear(in_features=896, out_features=128, bias=True)
      (o_proj): Linear(in_features=896, out_features=896, bias=False)
    )
    (mlp): Qwen2MLP(
      (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
      (up_proj): Linear(in_features=896, out_features=4864, bias=False)
      (down_proj): Linear(in_features=4864, out_features=896, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
    (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
  )
)

In [ ]:
def patch_hidden_state(source_hidden_states : tuple,batch_length:torch.Tensor, model=model,target_prompt = None) -> None:
    """
    source_hidden_states: Tuple[Tensor:Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model
    batch_length: Size([batch_size])

    target_prompt: str: the prompt the patched hidden states should be conditioned on
    
    """


    # # set a foward hook 
    # def patch_hidden_hook(module, input, output):
    #     """
    #     output: Size([batch_size, sequence_length, hidden_size])
    #     """

    #     print(output)
    #     # print(f' the shape of the output is  {output}')


    patched_results = {l : [] for l in range(len(source_hidden_states))}


    for layer_index, layer in enumerate(source_hidden_states):

        layer = layer[torch.arange(len(batch_length)),batch_length -1 ,:] # Size([batch_size, hidden_size])


        for i in range(layer.shape[0]):

            hs_to_be_patched = layer[i]

            # set a farward hook function
            def patch_hidden_hook(module, input, output):
                """
                input: Tuple[Tensor:Size([batch_size, sequence_length, hidden_size])]
                output: Tuple[Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model

                """
                # print(f' ')
                output[0][0,-1] = hs_to_be_patched

                if layer_index == 23:  # for debugging reason 
                    print(output[0][0,-1])


            # set a hook to the target layer
            hook_handle = model.model.layers[layer_index].register_forward_hook(patch_hidden_hook)

            with torch.no_grad():
                output_Target = model(**tokenizer(target_prompt,return_tensors='pt').to(device),output_hidden_states=True)
                target_logits = output_Target.logits[0][-1]
                patched_results[layer_index].append(target_logits)

            hook_handle.remove()


            if layer_index == 23:  # for debugging reason

                print(output_Target.hidden_states[23 + 1][0][-1])

    
    for k,v in patched_results.items():
        patched_results[k] = torch.stack(v)

    return patched_results



In [113]:
tokenizer(identity_token_prompts[0],return_tensors='pt',padding=True)['input_ids'].shape


torch.Size([1, 50])

In [114]:
def cal_precision_1(source, target,sources_length):
    """
    source: Size([batch_size, sequence_len, vocab_size])
    target: Dict(K: num_layer, V: Size([batch_size, vocab_size]))

    """
    
    source = source[torch.arange(len(sources_length)),sources_length -1 ,:] # Size([batch_size, vocab_size])

    for i in range(len(target)):
        print(torch.sum(torch.argmax(source,dim=-1) == torch.argmax(target[i],dim=-1),dim = 0))

In [116]:
for prompt in identity_token_prompts:

    # get the hidden satate of the prompt at the last positon 


    batch_size = 4
    for batch in DataLoader(validation_data, batch_size=batch_size, collate_fn=data_collator):
        # print(batch)
        batch.to(device)

        # run the first forward pass on the source prompt
        with torch.no_grad():
            # with autocast('cuda'):
            print("hello 123")
            model_outputs = model(**batch,output_hidden_states=True)
            print("heelo ")
            hidden_states = model_outputs.hidden_states[1:]
            reference_logits = model_outputs.logits # Size([batch_size, sequence_length, vocab_size])
            batch_length = torch.sum(batch['attention_mask'],dim=1)

        target_logtis = patch_hidden_state(hidden_states,batch_length,target_prompt=prompt) 


        
        cal_precision_1(reference_logits,target_logtis,batch_length)

        batch.to('cpu')
        del batch, model_outputs
        torch.cuda.empty_cache()
    
        break


    break

hello 123


OutOfMemoryError: CUDA out of memory. Tried to allocate 68.00 MiB. GPU 0 has a total capacity of 10.90 GiB of which 15.06 MiB is free. Including non-PyTorch memory, this process has 10.88 GiB memory in use. Of the allocated memory 10.22 GiB is allocated by PyTorch, and 515.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
! nvidia-smi

Fri Feb 28 00:12:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off |   00000000:3B:00.0 Off |                  N/A |
| 25%   45C    P5             15W /  250W |    8144MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# to check at the last layer if the logits are the same as the target logits 

SyntaxError: invalid syntax (483644964.py, line 3)